# Newton-Raphson fractal animation notebook

In [1]:
import os
import sys
import numpy as np
from tqdm import tqdm
from itertools import product
from joblib import Parallel, delayed

import matplotlib.pyplot as plt

from polynomial import Polynomial

from newton_engine import get_NR_colors
from newton_plots import NR_fractal_mx

In [2]:
N = 1024
n_steps = 50
n_frames = 90
ffmt = 'png'
fdpi = 100  

outdir = './out/frames-N{}-ns{}/'.format(N, n_steps)
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [3]:
# Polynomial to calculate
P = Polynomial(c=[1,0,0,1,-1,1])

# Predefined limits
gl_s = ((-1.5,1.5),(-1.5,1.5))
gl_e = ((-1.0092734515,-1.0092734495), (0.1473217440, 0.1473217460))

In [4]:
def NR_fractal_get_frames(gl_s, gl_e, n_frames=50):
  
  v_s = np.array(list(product(*gl_s)))
  v_e = np.array(list(product(*gl_e)))

  sg = np.sign(v_s - v_e)
  f_s = v_s - v_e
  f_e = np.zeros_like(v_s) + 1e-15 * sg

  v_coords = np.geomspace(f_s, f_e, n_frames) + v_e
  
  return v_coords

def NR_fractal_get_grid_lims(v_coords):
  
  # Shorten variable names
  v1, v2 = v_coords[:,0], v_coords[:,3]
  
  grid_lims =\
  tuple(
    tuple(
      ( (xmin, xmax),(ymin, ymax) )
    )
    for xmin, xmax, ymin, ymax in zip(v1[:,0], v2[:,0], v1[:,1], v2[:,1])
  )
  
  return grid_lims

In [5]:
def create_frame(i, P,
                 N, n_steps, gl,
                 outdir, ffmt, fdpi):
  X_n, _, _, _= NR_fractal_mx(P,
                              N=N, n_steps=n_steps,
                              grid_lim_x=gl[0],
                              grid_lim_y=gl[1])
  X = get_NR_colors(P, X_n).reshape((N, N, 4))

  fname = 'nrfractal_anim_frame-{:04d}.'.format(i)
  plt.imsave(fname=(outdir + fname + ffmt),
             arr=X, vmin=0, vmax=1,
             dpi=fdpi)

In [6]:
# Get the extents of the frames
v_coords = NR_fractal_get_frames(gl_s, gl_e, n_frames=n_frames)
grid_lims = NR_fractal_get_grid_lims(v_coords)
#del v_coords

In [7]:
%%time
_ = Parallel(n_jobs=4)(delayed(create_frame)(i, P,
                                             N, n_steps, gl,
                                             outdir, ffmt, fdpi) for i, gl in enumerate(tqdm(grid_lims)));

100%|███████████████████████████████████████████████████████████████████████████████| 90/90 [08:00<00:00,  5.34s/it]


CPU times: user 151 ms, sys: 51.3 ms, total: 203 ms
Wall time: 8min 55s


## Append frames

In [8]:
import imageio
from datetime import datetime

In [9]:
def create_anim(N : int, n_steps : int):
  dir_ = 'frames-N{}-ns{}/'.format(N, n_steps)
  name = 'NR_fractal-N{}-ns{}'.format(N, n_steps)

  path = './out/' + dir_
  outdir = './out/' + dir_ + 'anim/'
  if not os.path.exists(outdir):
    os.makedirs(outdir)

  # Gather path to appropriate images to concatenate
  print('[{0}] Reading in list of images to concatenate'.format(datetime.now().isoformat()), file=sys.stderr)
  file_list = []
  for file in os.listdir(path):
    if 'nrfractal' in file:
      complete_path = path + file
      file_list.append(complete_path)
  # Sort filenames
  file_list.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))

  # Define writer and create video
  writer = imageio.get_writer(outdir + '{}.mp4'.format(name),
                              codec='h264', bitrate='3500k',
                              format='mp4', fps=30)

  for i, im in enumerate(file_list):
    text = '[{0}] Appending image {1}...'.format(datetime.now().isoformat(), i)
    print(text, file=sys.stderr)
    writer.append_data(imageio.imread(im))
  writer.close()

In [10]:
create_anim(N=N, n_steps=n_steps)

[2021-10-28T16:23:56.976285] Reading in list of images to concatenate
[2021-10-28T16:23:56.978347] Appending image 0...
[2021-10-28T16:23:57.056328] Appending image 1...
[2021-10-28T16:23:57.093133] Appending image 2...
[2021-10-28T16:23:57.117467] Appending image 3...
[2021-10-28T16:23:57.145953] Appending image 4...
[2021-10-28T16:23:57.168773] Appending image 5...
[2021-10-28T16:23:57.192965] Appending image 6...
[2021-10-28T16:23:57.215398] Appending image 7...
[2021-10-28T16:23:57.233563] Appending image 8...
[2021-10-28T16:23:57.256062] Appending image 9...
[2021-10-28T16:23:57.279088] Appending image 10...
[2021-10-28T16:23:57.302885] Appending image 11...
[2021-10-28T16:23:57.330678] Appending image 12...
[2021-10-28T16:23:57.355921] Appending image 13...
[2021-10-28T16:23:57.379146] Appending image 14...
[2021-10-28T16:23:57.426663] Appending image 15...
[2021-10-28T16:23:57.446787] Appending image 16...
[2021-10-28T16:23:57.475114] Appending image 17...
[2021-10-28T16:23:57.4